In [ ]:
import pickle

import ray

import main

In [ ]:
try:
    with open('data/data.pickle', 'rb') as file:
        init_data = pickle.load(file)
except FileNotFoundError:
    init_data = None

In [ ]:
env_name = 'BreakoutNoFrameskip-v4'
agent_name = 'double'
data = init_data

In [ ]:
parallel_calls = 4
ray.init(num_cpus=parallel_calls, num_gpus=1)

batch_size = 64
n_steps = 2
init_sample_eps = 0.1  # 1 means random sampling

buffer = main.BUFFERS[agent_name](min_size=batch_size)
agent_object = main.AGENTS[agent_name]
agent_object = ray.remote(num_gpus=1/parallel_calls)(agent_object)
agents = [agent_object.remote(env_name,
                              buffer.table_name, buffer.server_port, buffer.min_size,
                              n_steps, data, make_sparse=False,
                              init_epsilon=init_sample_eps) for _ in range(parallel_calls)]

In [ ]:
eps = 0.1
futures = [agent.train_collect.remote(iterations_number=1000, epsilon=eps) for agent in agents]
outputs = ray.get(futures)